In [1]:
import xmltodict
import os
import json

top_key = "LcmsAnalyserResults"


dict_list = []

for fn in os.listdir("xmls"):
    if ".xml" in fn:
        fp = os.path.join("xmls", fn)
        with open(fp, "r") as f:
            d = xmltodict.parse(f.read())
        
        for key in d[top_key]:
            sub = d[top_key][key]
            fout = os.path.join(
                "jsons",
                key,
                fn.replace(".xml", ".json")
            )
            os.makedirs(os.path.dirname(fout), exist_ok=True)
            with open(fout, "w") as f:
                json.dump(sub, f, indent=4)

In [2]:
test_section = "SectionInfo"



In [4]:
from google.cloud import bigquery

client = bigquery.Client()
test_section = "SectionInfo"


In [35]:
from google.cloud import bigquery
import json

# Initialize BigQuery client
client = bigquery.Client()

test_section = "SectionInfo"
dataset_id = "trackpulse_test"
table_id = f"{dataset_id}.{test_section}"
schema_path = f"jsons/{test_section}/schema.json"

# Load schema from JSON file
with open(schema_path, 'r') as schema_file:
    schema = json.load(schema_file)

# Define dataset reference
dataset_ref = client.dataset(dataset_id)

# Create dataset if it doesn't exist
try:
    dataset = client.get_dataset(dataset_ref)
except Exception as e:
    print(f"Dataset {dataset_id} does not exist, creating...")
    dataset = bigquery.Dataset(dataset_ref)
    dataset = client.create_dataset(dataset)
    print(f"Dataset {dataset_id} created.")

# Define table reference
table_ref = dataset_ref.table(test_section)

# Create table if it doesn't exist
try:
    table = client.get_table(table_ref)
    # Update schema if it has changed
    if table.schema != schema:
        table.schema = schema
        table = client.update_table(table, ["schema"])
except Exception as e:
    print(f"Table {table_id} does not exist, creating... {e}")
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f"Table {table_id} created.")



JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 3 (char 5)

In [30]:
# Make a jsonl of the files in the directory
import json
import os

jsonl_path = f"jsons/{test_section}/data.jsonl"
with open(jsonl_path, "w") as f:
    for fn in os.listdir(f"jsons/{test_section}"):
        if fn.endswith(".json") and "schema" not in fn:
            source_fp = os.path.join(f"jsons/{test_section}", fn)
            print(f"Writing {source_fp}")
            with open(source_fp, "r") as sfp:
                din = json.load(sfp)
            f.write(json.dumps(din) + "\n")

Writing jsons/SectionInfo/LcmsResult_000000.json
Writing jsons/SectionInfo/LcmsResult_000001.json


In [31]:
# Upload data to BigQuery
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    schema=schema,
)

with open(jsonl_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for the job to complete

LoadJob<project=trackpulse, location=US, id=f419c2b3-8c58-4bb8-a6ce-56b031877f54>